In [ ]:
pip install anchor_exp

     |████████████████████████████████| 430kB 2.8MB/s 
     |████████████████████████████████| 276kB 11.9MB/s 
     |████████████████████████████████| 2.0MB 13.6MB/s 
  Created wheel for anchor-exp: filename=anchor_exp-0.0.1.0-cp36-none-any.whl size=434742 sha256=5f6978a7be0f4a440bf9e7274680a36e31d7b7deadf5843ad021a0a4f68ab02f
  Stored in directory: /root/.cache/pip/wheels/ff/b7/cd/7bfb36f4a01ff6b1509cd3432f8b208f455d232cee1079e309
  Created wheel for lime: filename=lime-0.2.0.0-cp36-none-any.whl size=284181 sha256=1bcf1bc5886cd37fd170676b5a7526aa8c99fe05702aa39cb28313fb5508f528
  Stored in directory: /root/.cache/pip/wheels/22/f2/ec/e5ebd07348b2b1ac722e91c2f549fcc220f7d5f25497a61232
Successfully built anchor-exp lime
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [ ]:
pip install spacy && python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=451ae39b2879e34fff08c787eccab0ca264c5fad5e687908970f7610395b3c97
  Stored in directory: /tmp/pip-ephem-wheel-cache-q7_mqn_5/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import time
import en_core_web_lg

In [ ]:
import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
import time
from sklearn.metrics import classification_report

In [ ]:
def load_polarity(path):
    data = []
    labels = []
    f_names = ['d.neg','d.pos']
    for (a, f) in enumerate(f_names):
        print(a,f)
        for line in open(f, 'rb'):
            try:
                line.decode('utf8')
            except:
                continue
            data.append(line.strip())
            labels.append(a)


    return data, labels

In [ ]:
data, labels = load_polarity('')


0 d.neg
1 d.pos


In [ ]:
print(len(data))
print(len(labels))
print(data[8])


18958
18958
b'"Tokyo Eyes" tells of a 17 year old Japanese girl who falls in like with a man being hunted by her big bro who is a cop. This lame flick is about 50% filler and 50% talk, talk, and more talk. You\'ll get to see the less than stellar cast of three as they talk on the bus, talk and play video games, talk and get a haircut, talk and walk and walk and talk, talk on cell phones, hang out and talk, etc. as you read subtitles waiting for something to happen. The thin wisp of a story is not sufficient to support a film with low end production value, a meager cast, and no action, no romance, no sex or nudity, no heavy drama...just incessant yadayadayada\'ing. (C-)'


## Data Cleaning

In [ ]:
random_state = None
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re, string, unicodedata
import inflect
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords as sw
sww = sw.words()
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean(text):
    clean_data = []
    for corpus in text:
        #print(corpus)
        if type(corpus) is bytes:
            corpus = corpus.decode('utf-8')
        #chnage word to lower case, we want to ignore capital or lower cases
        lower_doc = corpus.lower()
 
        #expan construction of words into it's full representation
        #expand_doc = expandContractions(lower_doc)
        
        #delete punctuation
        clean_doc = re.sub(r'[^\w\s]','',lower_doc)
        
        #split sentence into words
        tokenized_doc1 = word_tokenize(clean_doc)
        
        sww = sw.words() 
        new_corpus = [w for w in tokenized_doc1 if not w in sww]
        clean_data.append(new_corpus)
    return clean_data

In [ ]:
data_new = clean(data)
del data

In [ ]:
print(len(data_new))
print(len(labels))

18958
18958


In [ ]:
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data_new, labels, test_size=.2, random_state=42)
train, val, train_labels, val_labels = sklearn.model_selection.train_test_split(train, train_labels, test_size=.1, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

In [ ]:
print(train[0])

['true', 'story', 'great', 'pianist', 'jazz', 'singerlegend', 'ray', 'charles', 'oscar', 'bafta', 'golden', 'globe', 'winning', 'jamie', 'foxx', 'born', 'poor', 'african', 'americantown', 'went', 'blind', '7', 'years', 'old', 'skills', 'touch', 'hearing', 'would', 'later', 'life', 'would', 'lead', 'stardom', '1960s', 'accomplished', 'dream', 'selling', 'records', 'millions', 'leading', 'charts', 'songs', 'albums', 'story', 'showed', 'downfalls', 'including', 'separation', 'wife', 'child', 'affair', 'band', 'member', 'alcohol', 'use', 'going', 'prison', 'starring', 'regina', 'king', 'margie', 'hendricks', 'kerry', 'washington', 'bea', 'robinson', 'clifton', 'powell', 'jeff', 'brown', 'harry', 'lennix', 'joe', 'adams', 'bokeem', 'woodbine', 'fathead', 'newman', 'aunjanue', 'ellis', 'mary', 'ann', 'fisher', 'sharon', 'warren', 'aretha', 'robinson', 'cj', 'sanders', 'young', 'ray', 'robinson', 'curtis', 'armstrong', 'ahmet', 'ertegun', 'richard', 'schiff', 'jerry', 'wexler', 'great', 'stor

In [ ]:
vectorizer = CountVectorizer(min_df=1)

def to_str(text):
  text_str = []
  for line in text:
     text_str.append(" ".join(line))
  return text_str


train_str = to_str(train)
test_str = to_str(test)
val_str = to_str(val)
vectorizer.fit(train_str)

train_vectors = vectorizer.transform(train_str)
test_vectors = vectorizer.transform(test_str)
val_vectors = vectorizer.transform(val_str)


In [ ]:
print(train_vectors.shape)
print(test_vectors.shape)
print(val_vectors.shape)

(13649, 84540)
(3792, 84540)
(1517, 84540)


## MLP

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torch import Tensor
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_input = 84540
        num_classes = 2
        hidden_size1 = 256
        hidden_size2 = 256
        self.linear1 = nn.Linear(num_input, hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        hidden1 = self.linear1(x)
        nn.Dropout(0.4)
        activation1 = F.relu(hidden1)
        hidden2 = self.linear2(activation1)
        nn.Dropout(0.4)
        activation2 = F.relu(hidden2)
        out = self.linear2(activation2)
        ou t = F.log_softmax(out, dim=1)
        return out


In [ ]:
def prepare_batch(data, labels, size):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(data.shape[0]), size, replace=True)

    for i in random_index:
        random_inputs.append(data[i])
        random_labels.append(labels[i])
    return np.array(random_inputs), np.array(random_labels)

In [ ]:
#get the model
net = Net().to(device)

#learning rate
learning_rate=0.1

#calculate loss by function. The negative log likelihood loss.
criterion = nn.NLLLoss()

# Define the optimiser
# Pass the model parameters to be updated, and setup the learning rate when calling optim.SGD
# Please find the detailed information about the SGD optimiser in PyTorch (https://pytorch.org/docs/stable/optim.html).
optimiser = optim.SGD(net.parameters(), lr=learning_rate) 

#epochs
no_of_epochs = 200

# Every epoch, the model will be modified by the given learning rate
for epoch in range(no_of_epochs):  

    # get the input and label
    inputs, labels = prepare_batch(train_vectors.toarray(), train_labels, 300)
    inputs = torch.from_numpy(inputs).float().to(device)
    labels = torch.from_numpy(labels).long().to(device)
    
    net.train()

    optimiser.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimiser.step() # back propagation

    if epoch % 20 == 0:    # print every 100 epochs
        print('%d, loss: %.3f' %(epoch + 1, loss.item()))


1, loss: 5.588
21, loss: 1.402
41, loss: 0.841
61, loss: 0.876
81, loss: 0.576
101, loss: 0.578
121, loss: 0.539
141, loss: 0.422
161, loss: 0.479
181, loss: 0.333
201, loss: 0.230
221, loss: 0.491
241, loss: 0.191
261, loss: 0.457
281, loss: 0.174
301, loss: 0.224
321, loss: 0.145
341, loss: 0.361
361, loss: 0.192
381, loss: 0.145
401, loss: 0.150
421, loss: 0.124
441, loss: 0.521
461, loss: 0.273
481, loss: 0.123


In [ ]:
test_torch = torch.from_numpy(test_vectors.toarray()).float().to(device)

output = net(test_torch)
_, predicted = torch.max(output, 1)
predicted = Tensor.numpy(predicted)
print("Accuracy: %s" % np.mean(predicted == test_labels))
print(classification_report(test_labels, predicted))

Accuracy: 0.8850210970464135
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      2072
           1       0.86      0.89      0.88      1720

    accuracy                           0.89      3792
   macro avg       0.88      0.89      0.88      3792
weighted avg       0.89      0.89      0.89      3792



## Anchor

In [ ]:
nlp = en_core_web_lg.load()
#test_str = to_str(test)

In [ ]:
def predict_nn(texts):
    vec = vectorizer.transform(texts)
    x_torch = torch.from_numpy(vec.toarray()).float().to(device)
    output = net(x_torch)
    _, predicted = torch.max(output, 1)
    predicted = Tensor.numpy(predicted).tolist()
    return np.array(predicted)

In [ ]:
predict_nn([text])

array([1])

In [ ]:
idx = 0

explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=True)
np.random.seed(1)
text = test_str[idx]
pred = explainer.class_names[predict_nn([text])[idx]]
alternative =  explainer.class_names[1 - predict_nn([text])[0]]
print('Prediction: %s' % pred)
print('True label: %s' % 'positive' if test_labels[idx] == 1 else 'True label: %s' % 'negative')


Prediction: positive
True label: positive


In [ ]:
len(text)

466

In [ ]:
exp = explainer.explain_instance(text, predict_nn, threshold=0.90)

In [ ]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Anchor: situation AND bugs
Precision: 0.94

Examples where anchor applies and model predicts positive:

UNK sylvester cartoons made UNK pair UNK bugs bunny terms UNK tunes UNK UNK besi
UNK UNK cartoons made legendary UNK UNK bugs bunny terms looney tunes popularity
tweety UNK UNK UNK UNK pair UNK bugs UNK UNK UNK tunes UNK UNK UNK UNK UNK UNK s
tweety sylvester UNK UNK legendary pair UNK bugs UNK terms UNK tunes UNK br UNK 
UNK UNK cartoons UNK UNK UNK UNK bugs UNK UNK looney tunes UNK UNK UNK stock UNK
tweety sylvester UNK UNK legendary UNK second bugs UNK UNK UNK tunes UNK UNK UNK
UNK UNK UNK UNK legendary pair UNK bugs bunny UNK looney tunes popularitybr UNK 
UNK UNK UNK made UNK pair second bugs bunny terms UNK UNK popularitybr UNK UNK U
tweety UNK cartoons UNK legendary pair UNK bugs UNK UNK looney UNK UNK UNK besid
tweety sylvester UNK UNK legendary pair UNK bugs bunny terms UNK UNK UNK br besi

Examples where anchor applies and model predicts negative:

UNK UNK UNK made UNK UNK 

In [ ]:
# Sample text is too long to vias
exp.show_in_notebook()

IndexError: ignored

## LIME

In [ ]:
from lime import lime_text
from lime.lime_text import LimeTextExplainer

In [ ]:
#pip install lime

In [ ]:
 def findprob(test):

    test = vectorizer.transform(test)

    test = torch.from_numpy(test.toarray()).float().to(device)
    pred_outputs = net(test)
    out = pred_outputs.data.cpu().numpy()
    exp = np.exp(out)
    expsum = 0
    for i in range(len(exp[0])):
        expsum += exp[0][i]

    
    #prob = prob.astype('float')
    return exp/expsum

In [ ]:
class_names = ['neg', "pos"]
explainer_L = LimeTextExplainer(class_names=class_names)

In [ ]:
idx = 0
print('Document id: %d' % idx)
pred_outputs = net(test_torch[idx].reshape(-1, 84540))
predicted = torch.argmax(pred_outputs, 1)
int(predicted[0].data.cpu().numpy())
print('Predicted class =', class_names[int(predicted[0].data.cpu().numpy())])
print('True class: %s' % class_names[test_labels[idx]])

Document id: 0
Predicted class = pos
True class: pos


In [ ]:
exp_L = explainer_L.explain_instance(test_str[idx], findprob, num_features=5, top_labels = 1)

/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]


In [ ]:
exp_L.as_list()

[('sylvester', 0.1514226384370585),
 ('situation', 0.09157544000410336),
 ('pair', -0.08451584269305518),
 ('time', 0.07924365966418842),
 ('garbage', -0.07000775502614492)]

In [ ]:
#exp.show_in_notebook()

# Afinn


In [ ]:
pip install afinn

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp36-none-any.whl size=53452 sha256=7e4bf65eebba249f43f3825e42fbad2b62e14dc143d9e1af94ff6f02f76946f3
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


In [ ]:
from afinn import Afinn
from nltk.corpus import gutenberg
import textwrap
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
afinn = Afinn()
def afi(text, target, topk):
    dic = {}
    for w in text:  
        dic[w] = afinn.score(w)
    if target == 0:
        A = sorted(dic.items(), key = lambda x:x[1])
        return [A[i][0] for i in range(topk)]
    if target == 1:
        A = sorted(dic.items(), key = lambda x:x[1], reverse = True)
        print(A)
        return [A[i][0] for i in range(topk)]

In [ ]:
test_str[idx]

'tweety sylvester cartoons made legendary pair second bugs bunny terms looney tunes popularitybr br besides stock situations duo sylvester feeding garbage cans tawt taw putty tat line etc tweetys sos stars granny types supporting characters warner bros classics abundance enrich color flavor time bird aboard ocean liner gags extracted situation creative livelybr br day cartoon history friz freleng decided pair sylvester departed robert clampetts little yellow bird'

In [ ]:
idx = 0
te = word_tokenize(test_str[idx])
afi(te, test_labels[idx], 5)  

[('creative', 2.0), ('supporting', 1.0), ('aboard', 1.0), ('tweety', 0.0), ('sylvester', 0.0), ('cartoons', 0.0), ('made', 0.0), ('legendary', 0.0), ('pair', 0.0), ('second', 0.0), ('bugs', 0.0), ('bunny', 0.0), ('terms', 0.0), ('looney', 0.0), ('tunes', 0.0), ('popularitybr', 0.0), ('br', 0.0), ('besides', 0.0), ('stock', 0.0), ('situations', 0.0), ('duo', 0.0), ('feeding', 0.0), ('cans', 0.0), ('tawt', 0.0), ('taw', 0.0), ('putty', 0.0), ('tat', 0.0), ('line', 0.0), ('etc', 0.0), ('tweetys', 0.0), ('sos', 0.0), ('stars', 0.0), ('granny', 0.0), ('types', 0.0), ('characters', 0.0), ('warner', 0.0), ('bros', 0.0), ('classics', 0.0), ('abundance', 0.0), ('enrich', 0.0), ('color', 0.0), ('flavor', 0.0), ('time', 0.0), ('bird', 0.0), ('ocean', 0.0), ('liner', 0.0), ('gags', 0.0), ('extracted', 0.0), ('situation', 0.0), ('livelybr', 0.0), ('day', 0.0), ('cartoon', 0.0), ('history', 0.0), ('friz', 0.0), ('freleng', 0.0), ('decided', 0.0), ('departed', 0.0), ('robert', 0.0), ('clampetts', 0.0

['creative', 'supporting', 'aboard', 'tweety', 'sylvester']

In [ ]:
 test[idx]

['tweety',
 'sylvester',
 'cartoons',
 'made',
 'legendary',
 'pair',
 'second',
 'bugs',
 'bunny',
 'terms',
 'looney',
 'tunes',
 'popularitybr',
 'br',
 'besides',
 'stock',
 'situations',
 'duo',
 'sylvester',
 'feeding',
 'garbage',
 'cans',
 'tawt',
 'taw',
 'putty',
 'tat',
 'line',
 'etc',
 'tweetys',
 'sos',
 'stars',
 'granny',
 'types',
 'supporting',
 'characters',
 'warner',
 'bros',
 'classics',
 'abundance',
 'enrich',
 'color',
 'flavor',
 'time',
 'bird',
 'aboard',
 'ocean',
 'liner',
 'gags',
 'extracted',
 'situation',
 'creative',
 'livelybr',
 'br',
 'day',
 'cartoon',
 'history',
 'friz',
 'freleng',
 'decided',
 'pair',
 'sylvester',
 'departed',
 'robert',
 'clampetts',
 'little',
 'yellow',
 'bird']

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
gutenberg.sents('austen-sense.txt')

[['[', 'Sense', 'and', 'Sensibility', 'by', 'Jane', 'Austen', '1811', ']'], ['CHAPTER', '1'], ...]

In [ ]:
afinn = Afinn()
(test[idx].decode('utf-8'))
sentences = []
for line in test:
  sentences.append(line.decode('utf-8'))

#sentences = test.decode('utf-8')
print(sentences)
#scored_sentences = ((afinn.score(sent), sent) for sent in sentences)
#for sent in sentences:
#  print(sent)
scored_sentences
sorted_sentences = sorted(scored_sentences)
#print("\n".join(textwrap.wrap(sorted_sentences[0][1], 70)))

AttributeError: ignored

In [ ]:
sentences = (" ".join(wordlist) for wordlist in gutenberg.sents('austen-sense.txt'))
scored_sentences = ((afinn.score(sent), sent) for sent in sentences)
scored_sentences
sorted_sentences = sorted(scored_sentences)
sorted_sentences
#print("\n".join(textwrap.wrap(sorted_sentences[0][1], 70)))